<a href="https://colab.research.google.com/github/MangoDingo33/Study/blob/main/%ED%8A%B9%EA%B0%95/0926_AdversarialCapabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext tensorboard

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import models, transforms
from tqdm import tqdm

In [ ]:
trans = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

trans_train = transforms.Compose([transforms.Resize((224, 224)),
                                  transforms.RandomHorizontalFlip(p = 0.5),
                                  transforms.RandomRotation(degrees = (-90, 90)),
                                  transforms.RandomVerticalFlip(p = 0.5),
                                  transforms.ToTensor()])

In [ ]:
def save_checkpoint(model, filename):
  torch.save(model.state_dict(), filename)

train_set = torchvision.datasets.CIFAR10(
    root = './data', train = True, download = True, transform= trans_train)
test_set = torchvision.datasets.CIFAR10(
    root = './data', train = False, download= True, transform = trans)

trainloader = torch.utils.data.DataLoader(
    train_set, batch_size = 32, shuffle = True)
testloader = torch.utils.data.DataLoader(
    test_set, batch_size = 32, shuffle = False)

100%|██████████| 170498071/170498071 [00:13<00:00, 12182275.65it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
model = models.resnet50(pretrained = True)

num_classes = 10
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 50, eta_min = 0)

In [ ]:
h_acc = 0
n = 2

for epoch in range(n):
  loop = tqdm(enumerate(trainloader), total = len(trainloader))
  total_loss = 0
  total_correct = 0
  model.train()
  for i, data in loop:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(outputs, 1)

    train_loss = loss.item()
    train_correct = torch.sum(predicted == labels.data).item()

    loop.set_description(f"Epoch [{epoch + 1}/{n}]")
    loop.set_postfix(loss = train_loss, acc = train_correct / len(labels))

    total_loss += train_loss
    total_correct += train_correct

scheduler.step()

e_loss = total_loss / len(train_set)
e_acc = total_correct / len(train_set)

Epoch [2/2]: 100%|██████████| 1563/1563 [09:59<00:00,  2.61it/s, acc=0.562, loss=1.15]


In [ ]:
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))
model.eval()
with torch.no_grad():
  total_loss = 0
  total_correct = 0
  loop = tqdm(enumerate(testloader), total = len(testloader))
  for i, data in loop:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    _, predicted = torch.max(outputs, 1)
    c = (predicted == labels).squeeze()

    test_loss = loss.item()
    test_correct = torch.sum(predicted == labels.data).item()

    total_loss += test_loss
    total_correct += test_correct

    for i in range(len(labels)):
      label = labels[i]
      class_correct[label] += c[i].item()
      class_total[label] += 1

  e_loss = total_loss / len(test_set)
  e_acc = total_correct / len(test_set)

100%|██████████| 313/313 [00:41<00:00,  7.52it/s]


In [ ]:
if h_acc <= e_acc:
  save_checkpoint(model, '/content/drive/MyDrive/Colab Notebooks/Cifar_Best_resnet50.pth')
  h_acc = e_acc

print("fin")
print('best resnet50 Acc : ', h_acc)

fin
best resnet50 Acc :  0.8708
